# Ballbot Wall Pushing Task

In [ ]:
import Pkg; 

# Pkg.add("PyPlot")
# Pkg.add("RobotDynamics")
# Pkg.add("Ipopt")
# Pkg.add("MathOptInterface")
# Pkg.add("FileIO")
# Pkg.add("JLD2")
# Pkg.add("ECOS")
# Pkg.add("Convex")
# Pkg.add("FiniteDiff")
# Pkg.add("Plots")
# Pkg.add("TrajOptPlots")

Pkg.activate(@__DIR__); 
Pkg.instantiate()

In [ ]:
# Pkg.add("PyPlot")
# Pkg.add("RobotDynamics")
# Pkg.add("Ipopt")
# Pkg.add("MathOptInterface")
# Pkg.add("FileIO")
# Pkg.add("JLD2")
# Pkg.add("ECOS")
# Pkg.add("Convex")
# Pkg.add("FiniteDiff")
# Pkg.add("Plots")
# Pkg.add("TrajOptPlots")
ballbot = Ballbot_arms()

In [ ]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using ForwardDiff
#using PyPlot
using Plots
using RobotDynamics
using Ipopt
using TrajOptPlots
using GeometryBasics: HyperRectangle, Vec

import ECOS
import Convex as cvx
import MathOptInterface as MOI
using FileIO, JLD2;

import MeshCat as mc

In [ ]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","ballbot_model.jl"))

In [ ]:
ones(1,6)

In [ ]:
bb = Ballbot_arms()

vis = Visualizer()
render(vis)
delete!(vis)
mvis = MechanismVisualizer(bb.mech, URDFVisuals(urdf_arms), vis["test"])

In [ ]:
bb_wall = Ballbot_arms()
RigidBodyDynamics.bodies(bb_wall.mech)

In [ ]:
bb_arms = Ballbot_arms()
contact_state = MechanismState(bb_arms.mech)

q = zeros(19)
q[1] = -0.5 # move to a distance
q[3] = -0.10 # lean forward
q[6] = 0.785 # right shoulder
q[7] = -0.785 # left shoulder
q[12] = 1.5708 # right elbow 90 deg
q[13] = -1.5708 # left elbow 90 deg
RigidBodyDynamics.set_configuration!(contact_state, q)
RigidBodyDynamics.set_velocity!(contact_state, zeros(19))
world = RigidBodyDynamics.findbody(bb_arms.mech, "world")

right_end = RigidBodyDynamics.findbody(bb_arms.mech, "RArm7")
right_end_tf = RigidBodyDynamics.frame_definitions(right_end)[end].from
r_end_eff_pos = transform(contact_state, Point3D(right_end_tf, zero(SVector{3})), default_frame(world))

left_end = RigidBodyDynamics.findbody(bb_arms.mech, "LArm7")
left_end_tf = RigidBodyDynamics.frame_definitions(left_end)[end].from
l_end_eff_pos = transform(contact_state, Point3D(left_end_tf, zero(SVector{3})), default_frame(world))

@show r_end_eff_pos
@show l_end_eff_pos

In [ ]:
r_end_eff_pos.v[1]

In [ ]:
state = bb_wall.statecache[Float32]


In [ ]:
# world = RigidBodyDynamics.findbody(bb_wall.mech, "world")

# right_end = RigidBodyDynamics.findbody(bb_wall.mech, "RArm4")
# right_end_tf = RigidBodyDynamics.frame_definitions(right_end)[end].from
# r_end_eff_pos = transform(state, Point3D(right_end_tf, zero(SVector{3})), default_frame(world))

# left_end = RigidBodyDynamics.findbody(bb_wall.mech, "LArm4")
# left_end_tf = RigidBodyDynamics.frame_definitions(left_end)[end].from
# l_end_eff_pos = transform(state, Point3D(left_end_tf, zero(SVector{3})), default_frame(world))

# @show r_end_eff_pos
# @show l_end_eff_pos

In [ ]:
# infrasructures functions

function get_endeffector_poses(model::Ballbot, q::AbstractVector{T}) where {T} 
    world1 = RigidBodyDynamics.findbody(model.mech, "world")
    state = model.statecache[T]
    # Convert from state ordering to the ordering of the mechanism
    #set_configuration!(state, q)
#     println("state.q is: ", size(state.q))
    copyto!(state.q, q)
    right_end_func = RigidBodyDynamics.findbody(model.mech, "RArm7")
    right_end_tf_func = RigidBodyDynamics.frame_definitions(right_end_func)[end].from
    left_end_func = RigidBodyDynamics.findbody(model.mech, "LArm7")
    left_end_tf_func = RigidBodyDynamics.frame_definitions(left_end_func)[end].from
    
    r_end_eff_pos = transform(state, Point3D(right_end_tf_func, zero(SVector{3})), default_frame(world1))
    l_end_eff_pos = transform(state, Point3D(left_end_tf_func, zero(SVector{3})), default_frame(world1))
    return r_end_eff_pos.v, l_end_eff_pos.v
end

function momentum_function(model::Ballbot, q::AbstractVector{T}) where {T} 
    state = model.statecache[T]
    res = model.dyncache[T]
    # Convert from state ordering to the ordering of the mechanism
    copyto!(state.q, q)
    #set_configuration!(state, q)
    out = momentum_matrix(state)
    return out.angular
end

function com_function(model::Ballbot, q::AbstractVector{T}) where {T} 
    state = model.statecache[T]
    res = model.dyncache[T]
    #out = model.momentum_m
    # Convert from state ordering to the ordering of the mechanism
    copyto!(state.q, q)
    #set_configuration!(state, q)
    out = center_of_mass(state)
    return out.v
end

function get_normal_tangential(F, n)
    Fn = dot(F, n)
    Ft = F-Fn*n
    return Fn, Ft
end
function skew(ω::Vector{T}) where {T}
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

function create_idx(nq,N) 
    nq̇ = nq̈ = nq
    nh = nḣ = nr = nṙ = nr̈ = nF = nτ = ntd = 3
    nd = 6
    nfriction = 1
   
    # Our Z vector is [x0, u0, x1, u1, …, xN]
    nzi = nq + nq̇ + nq̈ + nr + nṙ + nr̈ + nh + nḣ + 3*nF + 3*nτ + 2*nd
    nz = N * nzi
    
    q = [(i - 1) * nzi .+ (1 : nq) for i = 1:N]
    q̇ = [(i - 1) * nzi .+ ((nq + 1) : (2*nq)) for i = 1:N]
    q̈ = [(i - 1) * nzi .+ ((2*nq + 1) : (3*nq)) for i = 1:N]
    r = [(i - 1) * nzi .+ ((3*nq + 1) : (3*nq + nr)) for i = 1:N]
    ṙ = [(i - 1) * nzi .+ ((3*nq + nr + 1) : (3*nq + 2*nr)) for i = 1:N]
    r̈ = [(i - 1) * nzi .+ ((3*nq + 2*nr + 1) : (3*nq + 3*nr)) for i = 1:N]
    h = [(i - 1) * nzi .+ ((3*nq + 3*nr + 1) : (3*nq + 3*nr + nh)) for i = 1:N]
    ḣ = [(i - 1) * nzi .+ ((3*nq + 3*nr + nh + 1) : (3*nq + 3*nr + 2*nh)) for i = 1:N]
    Fb = [(i - 1) * nzi .+ ((3*nq + 3*nr + 2*nh + 1) : (3*nq + 3*nr + 2*nh + nF)) for i = 1:N]
    FL = [(i - 1) * nzi .+ ((3*nq + 3*nr + 2*nh + nF + 1) : (3*nq + 3*nr + 2*nh + 2*nF)) for i = 1:N]
    FR = [(i - 1) * nzi .+ ((3*nq + 3*nr + 2*nh + 2*nF + 1) : (3*nq + 3*nr + 2*nh + 3*nF)) for i = 1:N]
    τb = [(i - 1) * nzi .+ ((3*nq + 3*nr + 2*nh + 3*nF + 1) : (3*nq + 3*nr + 2*nh + 3*nF + nτ)) for i = 1:N]
    τL = [(i - 1) * nzi .+ ((3*nq + 3*nr + 2*nh + 3*nF + nτ + 1) : (3*nq + 3*nr + 2*nh + 3*nF + 2*nτ)) for i = 1:N]
    τR = [(i - 1) * nzi .+ ((3*nq + 3*nr + 2*nh + 3*nF + 2*nτ + 1) : (3*nq + 3*nr + 2*nh + 3*nF + 3*nτ)) for i = 1:N]
    dL  = [(i - 1) * nzi .+ ((3*nq + 3*nr + 2*nh + 3*nF + 3*nτ + 1) : (3*nq + 3*nr + 2*nh + 3*nF + 3*nτ + nd)) for i = 1:N]
    dR  = [(i - 1) * nzi .+ ((3*nq + 3*nr + 2*nh + 3*nF + 3*nτ + nd + 1) : (3*nq + 3*nr + 2*nh + 3*nF + 3*nτ + 2*nd)) for i = 1:N]
    # Constraint indexing when stacked up   
    nci = nr̈ + nh + nḣ + nh + nr + nṙ + nr + nq + nq̇ + 2*ntd
          
    dcr̈ = [(i - 1) * (nci) .+ (1 : nr̈) for i = 1:N]
    dch = [(i - 1) * (nci) .+ ((nr̈ + 1) : (nr̈ + nh)) for i = 1:N]
    dcḣ = [(i - 1) * (nci) .+ ((nr̈ + nh + 1) : (nr̈ + 2*nh)) for i = 1:N]
    jcr = [(i - 1) * (nci) .+ ((nr̈ + 2*nh + 1) : (2*nr̈ + 2*nh)) for i = 1:N]
    
    cch = [(i - 1) * (nci) .+ ((2*nr̈ + 2*nh + 1) : (2*nr̈ + 3*nh)) for i = 1:(N - 1)]
    ccr = [(i - 1) * (nci) .+ ((2*nr̈ + 3*nh + 1) : (3*nr̈ + 3*nh)) for i = 1:(N - 1)]
    ccṙ = [(i - 1) * (nci) .+ ((3*nr̈ + 3*nh + 1) : (4*nr̈ + 3*nh)) for i = 1:(N - 1)]
    ccq = [(i - 1) * (nci) .+ ((4*nr̈ + 3*nh + 1) : (4*nr̈ + 3*nh + nq)) for i = 1:(N - 1)]
    ccq̇ = [(i - 1) * (nci) .+ ((4*nr̈ + 3*nh + nq + 1) : (4*nr̈ + 3*nh + 2*nq)) for i = 1:(N - 1)]
    
        
    gctdL = [(i - 1) * (nci) .+ ((4*nr̈ + 3*nh + 2*nq + 2*nd + 2*nfriction + 1) : (4*nr̈ + 3*nh + 2*nq + 2*nd + 2*nfriction+ntd)) for i = 1:(N - 1)] # equality constriaint for d
    gctdR = [(i - 1) * (nci) .+ ((4*nr̈ + 3*nh + 2*nq + 2*nd + 2*nfriction+ntd + 1) : (4*nr̈ + 3*nh + 2*nq + 2*nd + 2*nfriction+2*ntd)) for i = 1:(N - 1)] # equality constriaint for d
    
    
    nciter = nci * (N - 1) + (nr + nr̈ + nh + nḣ)
    ncfix = nq + nq + nq̇ + nq̈ + nḣ + nh #+ 2*nd + 2*nfriction + 2*ntd
    println(nciter)
    
    icq = nciter .+ (1 : nq)
    gcq = nciter .+ ((nq + 1) : (2*nq))
    icq̇ = nciter .+ ((2*nq + 1) : (3*nq))
    icq̈ = nciter .+ ((3*nq + 1) : (4*nq))
    gcḣ = nciter .+ ((4*nq + 1) : (4*nq + nh))
    gch = nciter .+ ((4*nq + nh + 1) : (4*nq + 2*nh))
    
    
    
    nc = nciter + ncfix
    
    ncineqi = 2*nfriction
    ncineq = ncineqi * (N)
    # debug: prob just gonna use state LU for this one
#     gcdL =[(i - 1) * (ncineqi) .+ (1 : nd) for i = 1:(N - 1)] # inequality constraint for d >= 0
#     gcdR = [(i - 1) * (ncineqi) .+ ((nd + 1) : (2*nd)) for i = 1:(N - 1)] # inequality constraint for d >= 0
    
    gcfL = [(i - 1) * (ncineqi) .+ ((1) : (nfriction)) for i = 1:(N)] # inequality constraint for slipping and friction
    gcfR = [(i - 1) * (ncineqi) .+ ((nfriction + 1) : (2*nfriction)) for i = 1:(N)] # inequality constraint for slipping and friction

    
    return (q=q,q̇=q̇,q̈=q̈,r=r,ṙ=ṙ,r̈=r̈,h=h,ḣ=ḣ,Fb=Fb,τb=τb,FL=FL,τL=τL,FR=FR,τR=τR, dL=dL, dR=dR,
            dcr̈=dcr̈,dch=dch,dcḣ=dcḣ,cch=cch,ccr=ccr,ccṙ=ccṙ,jcr=jcr,ccq=ccq,ccq̇=ccq̇,
            icq=icq,gcq=gcq,icq̇=icq̇,icq̈=icq̈,gcḣ=gcḣ,
            nzi=nzi,N=N,nz=nz,nc=nc,nq=nq,nq̇=nq̇,nq̈=nq̈,nr=nr,nṙ=nṙ,nr̈=nr̈,nh=nh,nḣ=nḣ,nF=nF,nτ=nτ, nd=nd,gch=gch, gcfL=gcfL,gctdL=gctdL, gcfR=gcfR,gctdR=gctdR,ncineq=ncineq)
end

In [3]:
#centroidal constraints

function centroidal_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    r_end_eff_pos = params.r_end_eff_pos
    l_end_eff_pos = params.l_end_eff_pos
    idx, N, dt = params.idx, params.N, params.dt
    qic, q̇ic, q̈ic = params.qic, params.q̇ic, params.q̈ic
    g, m = params.g, params.m
    model = params.model1
    I_float = Matrix{Float64}(I, 3, 3)
    I_I = hcat(I_float,I_float)
    c = zeros(eltype(Z), idx.nc)
    n = params.n
    
    
    #println("init")
    for i = 2:N
        # Configuration and derivatives
        qi = Z[idx.q[i]]
        q̇i = Z[idx.q̇[i]]
        q̈i = Z[idx.q̈[i]]
        qimn = Z[idx.q[i-1]]
        q̇imn = Z[idx.q̇[i-1]]
        q̈imn = Z[idx.q̈[i-1]]
        # COM and derivatives
        ri = Z[idx.r[i]]
        ṙi = Z[idx.ṙ[i]]
        r̈i = Z[idx.r̈[i]]
        rimn = Z[idx.r[i-1]]
        ṙimn = Z[idx.ṙ[i-1]]
        # Angular momentum and derivative
        hi = Z[idx.h[i]]
        ḣi = Z[idx.ḣ[i]]
        himn = Z[idx.h[i-1]]
        ḣimn = Z[idx.ḣ[i-1]]
        #println("Contact point, force and torque")
        # Contact point, force and torque
        cbi = [qi[1:2]; 0]
        cRi, cLi = get_endeffector_poses(model, qi)
        Fbi = Z[idx.Fb[i]]
        τbi = Z[idx.τb[i]]
        FLi = Z[idx.FL[i]]
        τLi = Z[idx.τL[i]]
        FRi = Z[idx.FR[i]]
        τRi = Z[idx.τR[i]]
        contactL = false
        contactR = false
        if cRi[2] >= r_end_eff_pos.v[2]
            contactR = true
        end
        if cLi[2] >= l_end_eff_pos.v[2]
            contactL = true
        end
        # Centroidal dynamics
        if (contactL && contactR) # TODO: how to make sure it is in contact????
            c[idx.dcr̈[i]] .= Fbi + FLi + FRi + m*g - m*r̈i
            c[idx.dcḣ[i]] .= skew(cbi-ri)*Fbi + τbi + skew(cLi-ri)*FLi + τLi + skew(cRi-ri)*FRi + τRi - ḣi
            
            # friction constraint
            FLni, FLti = get_normal_tangential(FLi, n)
            FRni, FRti = get_normal_tangential(FRi, n)
            c[idx.gctdL] .= FLti - I_I*Z[idx.dL[i]]
            c[idx.gctdR] .= FRti - I_I*Z[idx.dR[i]]
        elseif (contactL && !contactR)
            c[idx.dcr̈[i]] .= Fbi + FLi + m*g - m*r̈i
            c[idx.dcḣ[i]] .= skew(cbi-ri)*Fbi + τbi + skew(cLi-ri)*FLi + τLi - ḣi
            
            # friction constraint
            FLni, FLti = get_normal_tangential(FLi, n)
            c[idx.gctdL] .= FLti - I_I*Z[idx.dL[i]]
        elseif (contactR && !contactL)
            c[idx.dcr̈[i]] .= Fbi + FRi + m*g - m*r̈i
            c[idx.dcḣ[i]] .= skew(cbi-ri)*Fbi + τbi + skew(cRi-ri)*FRi + τRi - ḣi
                    
            # friction constraint
            FRni, FRti = get_normal_tangential(FRi, n)
            c[idx.gctdR] .= FRti - I_I*Z[idx.dR[i]]
        else
            c[idx.dcr̈[i]] .= Fbi + m*g - m*r̈i
            #println("Centroidal dynamics: 2")
            c[idx.dcḣ[i]] .= skew(cbi-ri)*Fbi + τbi - ḣi
        end
        c[idx.dch[i]] .= momentum_function(model, qi)*q̇i - hi
        
        #println("Collocation constraints")
        
        # Collocation constraints
        c[idx.ccq[i-1]] .= qi - qimn - q̇i*dt
        c[idx.ccq̇[i-1]] .= q̇i - q̇imn - q̈i*dt
        c[idx.cch[i-1]] .= hi - himn - ḣi*dt
        c[idx.ccr[i-1]] .= ri - rimn - (ṙi + ṙimn)*dt/2
        c[idx.ccṙ[i-1]] .= ṙi - ṙimn - r̈i*dt
        #println("Joint configuration constraint")
        
        # Joint configuration constraint
        c[idx.jcr[i]] .= com_function(model, qi) - ri
        
        
        # friction constraint
#         Fn, Fti = get_normal_tangential(F, n)
#         I_float = Matrix{Float64}(I, 3, 3)
#         I_I = hcat(I_float,I_float)
#         c[idx.gctdL] .=  - I_I*Z[idx.dL[i]]
    #println("Initial and goal constraints")
    
    c1 = [Z[idx.q[1]][1]; Z[idx.q[1]][2]; 0]
    c[idx.jcr[1]] .= com_function(model, Z[idx.q[1]]) - Z[idx.r[1]]

        
    # Initial and goal constraints
    c[idx.icq] .= qic - Z[idx.q[1]]
#     c[idx.gcq] .= qgoal - Z[idx.q[end]] 
#     c[idx.icq̇] .= q̇ic - Z[idx.q̇[1]]
#     c[idx.icq̈] .= q̈ic - Z[idx.q̈[1]]
#     c[idx.gch] .= Z[idx.h[end]]
#     c[idx.gcḣ] .= Z[idx.ḣ[end]]
    
    end
    
    ri = Z[idx.r[1]]
    ṙi = Z[idx.ṙ[1]]
    r̈i = Z[idx.r̈[1]]
    hi = Z[idx.h[1]]
    ḣi = Z[idx.ḣ[1]]
    # initial contact friction and centroidal momentum constraints, doesn't really matter for now but put here anyways
    qi = Z[idx.q[1]]
    cbi = [qi[1:2]; 0]
    cRi, cLi = get_endeffector_poses(model, qi)
    Fbi = Z[idx.Fb[1]]
    τbi = Z[idx.τb[1]]
    FLi = Z[idx.FL[1]]
    τLi = Z[idx.τL[1]]
    FRi = Z[idx.FR[1]]
    τRi = Z[idx.τR[1]]
    contactL = false
    contactR = false
    if cRi[2] >= r_end_eff_pos.v[2]
        contactR = true
    end
    if cLi[2] >= l_end_eff_pos.v[2]
        contactL = true
    end
    if (contactL && contactR) # TODO: how to make sure it is in contact????
        c[idx.dcr̈[1]] .= Fbi + FLi + FRi + m*g - m*r̈i
        c[idx.dcḣ[1]] .= skew(cbi-ri)*Fbi + τbi + skew(cLi-ri)*FLi + τLi + skew(cRi-ri)*FRi + τRi - ḣi
            
        # friction constraint
        FLni, FLti = get_normal_tangential(FLi, n)
        FRni, FRti = get_normal_tangential(FRi, n)
        c[idx.gctdL] .= FLti - I_I*Z[idx.dL[1]]
        c[idx.gctdR] .= FRti - I_I*Z[idx.dR[1]]
    elseif (contactL && !contactR)
        c[idx.dcr̈[1]] .= Fbi + FLi + m*g - m*r̈i
        c[idx.dcḣ[1]] .= skew(cbi-ri)*Fbi + τbi + skew(cLi-ri)*FLi + τLi - ḣi
            
        # friction constraint
        FLni, FLti = get_normal_tangential(FLi, n)
        c[idx.gctdL] .= FLti - I_I*Z[idx.dL[1]]
    elseif (contactR && !contactL)
        c[idx.dcr̈[1]] .= Fbi + FRi + m*g - m*r̈i
        c[idx.dcḣ[1]] .= skew(cbi-ri)*Fbi + τbi + skew(cRi-ri)*FRi + τRi - ḣi
                    
        # friction constraint
        FRni, FRti = get_normal_tangential(FRi, n)
        c[idx.gctdR] .= FRti - I_I*Z[idx.dR[1]]
     else
        c[idx.dcr̈[1]] .= Fbi + m*g - m*r̈i
        #println("Centroidal dynamics: 2")
        c[idx.dcḣ[1]] .= skew(cbi-ri)*Fbi + τbi - ḣi
    end
    
    
    return c
end

function centroidal_inequality_constraint(params, Z)
    μ = params.μ
    r_end_eff_pos = params.r_end_eff_pos
    l_end_eff_pos = params.l_end_eff_pos
    idx, N, dt = params.idx, params.N, params.dt
    qic, q̇ic, q̈ic = params.qic, params.q̇ic, params.q̈ic
    g, m = params.g, params.m
    model = params.model1
    c = zeros(eltype(Z), idx.ncineq)
    e = ones(6)
    #friction cone ineq constraints
    I_float = Matrix{Float64}(I, 3, 3)
    I_I = hcat(I_float,I_float)
    for i = 1:N
        qi = Z[idx.q[i]]
        cbi = [qi[1:2]; 0]
        cRi, cLi = get_endeffector_poses(model, qi)
        FLi = Z[idx.FL[i]]
        τLi = Z[idx.τL[i]]
        FRi = Z[idx.FR[i]]
        τRi = Z[idx.τR[i]]
        dLi = Z[idx.dL[i]]
        dRi = Z[idx.dR[i]]
        contactL = false
        contactR = false
        if cRi[2] >= r_end_eff_pos.v[2]
            contactR = true
        end
        if cLi[2] >= l_end_eff_pos.v[2]
            contactL = true
        end
        if (contactL && contactR) # TODO: how to make sure it is in contact????
            FLni, FLti = get_normal_tangential(FLi, n)
            FRni, FRti = get_normal_tangential(FRi, n)
            c[idx.gcfL[i]] = e'*dLi - μ*FLni
            c[idx.gcfR[i]] = e'*dRi - μ*FRni
        elseif (contactL && !contactR)
            FLni, FLti = get_normal_tangential(FLi, n)
            FRni, FRti = get_normal_tangential(FRi, n)
            c[idx.gcfL[i]] = e'*dLi - μ*FLni

        elseif (contactR && !contactL)
            FLni, FLti = get_normal_tangential(FLi, n)
            FRni, FRti = get_normal_tangential(FRi, n)
            c[idx.gcfR[i]] = e'*dRi - μ*FRni
        end
    end
    
    return c
end
        
function centroidal_inequality_constraint_empty(params, Z)
    return zeros(eltype(Z), 0)
end

centroidal_inequality_constraint_empty (generic function with 1 method)

Obtain a valid wall pushing pose

In [ ]:
# bb_arms = Ballbot_arms()
# contact_state = MechanismState(bb_arms.mech)

# q = zeros(9)
# q[1] = -0.5 # move to a distance
# q[3] = -0.10 # lean forward
# q[6] = 0.785 # right shoulder
# q[7] = -0.785 # left shoulder
# q[8] = 1.5708 # right elbow 90 deg
# q[9] = -1.5708 # left elbow 90 deg
# RigidBodyDynamics.set_configuration!(contact_state, q)
# RigidBodyDynamics.set_velocity!(contact_state, zeros(9))
# world = RigidBodyDynamics.findbody(bb_arms.mech, "world")

# right_end = RigidBodyDynamics.findbody(bb_arms.mech, "RArm4")
# right_end_tf = RigidBodyDynamics.frame_definitions(right_end)[end].from
# r_end_eff_pos = transform(contact_state, Point3D(right_end_tf, zero(SVector{3})), default_frame(world))

# left_end = RigidBodyDynamics.findbody(bb_arms.mech, "LArm4")
# left_end_tf = RigidBodyDynamics.frame_definitions(left_end)[end].from
# l_end_eff_pos = transform(contact_state, Point3D(left_end_tf, zero(SVector{3})), default_frame(world))

# @show r_end_eff_pos
# @show l_end_eff_pos


Visualize what a wall pushing pose looks like

In [ ]:
bb_wall = Ballbot_wall(vec(l_end_eff_pos.v), vec(r_end_eff_pos.v))

state = MechanismState(bb_wall.mech)

RigidBodyDynamics.set_configuration!(state, q)
RigidBodyDynamics.set_velocity!(state, zeros(9))
ts, qs, vs = simulate(state, 3., Δt = 1e-2);


In [ ]:
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);

## Trajectory Optimization for Wall Pushing

We use DIRCOL to find a feasible wall-pushing trajectory.

### Dynamics

There are two kind of dynamics modes: contact and non contact.

We schedule the following contact sequence:

$$
\begin{align} 
\mathcal{M}_1 &= \{1\text{:}20,31\text{:}50\} \\
\mathcal{M}_2 &= \{21\text{:}30\}
\end{align}
$$

where $\mathcal{M}_1$ contains the time steps when the BallBot is in free space, and $\mathcal{M}_2$ contains the time steps when the BallBot is in contact with the wall. The jump map sets $\mathcal{J}_1$ and $\mathcal{J}_2$ are the indices where the mode of the next time step is different than the current, i.e. $\mathcal{J}_i \equiv \{k+1 \notin \mathcal{M}_i \; | \; k \in \mathcal{M}_i\}$. We can write these out explicitly as the following:

$$
\begin{align} 
\mathcal{J}_1 &= \{20\} \\
\mathcal{J}_2 &= \{30\}
\end{align}
$$

Both dynamics categories share the same dynamics function `ballbot_dynamics`. However, different ballbot models should be passed to it (generated from `Ballbot_arms()` for free space or `Ballbot_wall()` for when against the wall).

In [ ]:

function ballbot_dynamics(model::Ballbot, x::AbstractVector{T1}, u::AbstractVector{T2}) where {T1,T2} 
    T = promote_type(T1,T2)
    state = model.statecache[T]
    res = model.dyncache[T]
    ball_radius = 0.106
    # Convert from state ordering to the ordering of the mechanism
    copyto!(state, x)
    τ = [u[1:2] ./ ball_radius; 0.0; 0.0; u[3:end]]
        
    dynamics!(res, state, τ)
    q̇ = res.q̇
    v̇ = res.v̇
    return [q̇; v̇]
end

function jump1_map(x)
    # elbows experiences inelastic collision
    xn = [x[1:16]; 0.0; 0.0]
    return xn
end

# function jump2_map(x)
#     # moving into free space. nothing special.
#     return xn
# end

In [ ]:
function hermite_simpson(params::NamedTuple, model1::Ballbot, model2::Ballbot, x1::Vector, x2::Vector, u, dt::Real)::Vector
    # Input hermite simpson implicit integrator residual
    f1 = ballbot_dynamics(model1, x1, u)
    f2 = ballbot_dynamics(model2, x2, u)
    xm = 0.5*(x1 + x2) + (dt/8.0)*(f1 - f2)
    fm = ballbot_dynamics(model1, xm, u)
    r = x1 + (dt/6.0)*(f1 + 4*fm + f2) - x2
    return r
end

function rk4(params::NamedTuple, model::Ballbot, x::Vector, u, dt::Float64)
    # Vanilla RK4
    k1 = dt*ballbot_dynamics(model, x, u)
    k2 = dt*ballbot_dynamics(model, x + k1/2, u)
    k3 = dt*ballbot_dynamics(model, x + k2/2, u)
    k4 = dt*ballbot_dynamics(model, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

function momentum_cost(params, ḣ)
    return 0.5*(ḣ)'*params.Qh*(ḣ)
end

function state_accel_cost(params, a)
    return 0.5*(a)'*params.Qa*(a)
end

function com_stability_cost(params,ps,com)
    return 0.5*(com[1:2]-ps)'*params.Qcom*(com[1:2]-ps)
end

function goal_vel_cost(params,vs)
    return 0.5*(vs-params.vs_g)'*params.Qvs*(vs-params.vs_g)
end

function ball_pos_cost(params,ps, Qps) # ps = x[1:2]
    return 0.5*(ps-params.ps_g)'*Qps*(ps-params.ps_g)
end

function stage_cost(params,qk, q̇k, q̈k, ḣk, k)
    return ball_pos_cost(params, qk[1:2],params.Qps) + momentum_cost(params, ḣk) + state_accel_cost(params, q̈k)
end

function terminal_cost(params,qk, q̇k, hk, ḣk)
    Qv = params.Qfv
    Qh = params.Qfh
    Qḣ = params.Qfḣ
    xref = params.Xref
    return goal_vel_cost(params, q̇k[1:2])+ball_pos_cost(params, qk[1:2],params.Qfps)+0.5*((ḣk)'*Qḣ*(ḣk) + (hk)'*Qh*(hk))
end

function ballbot_cost(params::NamedTuple, Z::Vector)::Real
    idx, N = params.idx, params.N 
    # Stage cost
    J = 0 
    for k = 1:(N-1)
        qk = Z[idx.q[k]]
        q̇k = Z[idx.q̇[k]]
        q̈k = Z[idx.q̈[k]]
        ḣk = Z[idx.ḣ[k]]
        #ui = Z[idx.u[i]]
       
        J += stage_cost(params, qk, q̇k, q̈k, ḣk, k)
    end
    
    # Terminal cost
    q̇end = Z[idx.q̇[end]]
    qend = Z[idx.q[end]]
    hend = Z[idx.h[end]]
    ḣend = Z[idx.ḣ[end]]
    J += terminal_cost(params, qend, q̇end, hend, ḣend)
    return J 
end

function ballbot_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    M1, M2 = params.M1, params.M2
    J1, J2 = params.J1, params.J2
    idx, N = params.idx, params.N 
    xg = params.xg
    xic = params.xic
    xcgs = params.xcgs # contact goal constraint
    # Return all of the equality constraints

    c_dynamics = ballbot_dynamics_constraints(params, Z)

    c_contact = zeros(eltype(Z), length(xcgs) * 9)
    for i=1:length(xcgs)
        i_start = (i-1) * 9 + 1
        i_end = i * 9
        c_contact[i_start:i_end] = Z[idx.x[J1[i]][1:9]] - xcgs[i]
    end

    # c_unused = zeros(eltype(Z), 3 * N)
    # for i=1:N
    #     i_start = (i-1) * 3 + 1
    #     i_end = i * 3
    #     c_unused[i_start:i_end] = Z[idx.x[i][[2; 4; 5]]]
    # end

    # c_parallel = zeros(eltype(Z), 2 * N)
    # for i=1:N
    #     xi = Z[idx.x[1]]
    #     c_parallel[i*2-1] = xi[6] + xi[7]
    #     c_parallel[i*2] = xi[8] + xi[9]
    # end

    # c_end = zeros(eltype(Z), 2 * N)
    # contact_state = MechanismState{eltype(Z)}(params.model2.mech)
    # for i=1:N
    #     xi = Z[idx.x[1]]
    #     RigidBodyDynamics.set_configuration!(contact_state, xi[1:9])
    #     RigidBodyDynamics.set_velocity!(contact_state, zeros(9))
    #     world = RigidBodyDynamics.findbody(params.model2.mech, "world")

    #     right_end = RigidBodyDynamics.findbody(params.model2.mech, "RArm4")
    #     right_end_tf = RigidBodyDynamics.frame_definitions(right_end)[end].from
    #     r_pos = transform(contact_state, Point3D(right_end_tf, zero(SVector{3})), default_frame(world))

    #     left_end = RigidBodyDynamics.findbody(params.model2.mech, "LArm4")
    #     left_end_tf = RigidBodyDynamics.frame_definitions(left_end)[end].from
    #     l_pos = transform(contact_state, Point3D(left_end_tf, zero(SVector{3})), default_frame(world))
        
    #     # c_end[i*6-5:i*6-3] .= r_pos.v - r_end_eff_pos.v
    #     # c_end[i*6-2:i*6] .= l_pos.v - l_end_eff_pos.v
    #     c_end[i*2-1] .= r_pos.v[1] - r_end_eff_pos.v[1]
    #     c_end[i*2] .= l_pos.v[1] - l_end_eff_pos.v[1]
    # end

    # c = [c_dynamics; Z[idx.x[1]] - xic; Z[idx.x[N]] - xg; c_end]
#     c = [c_dynamics; Z[idx.x[1]] - xic; Z[idx.x[N]] - xg; c_contact]
    c = centroidal_equality_constraint(params, Z)
    return c
end

function ballbot_inequality_constraint(params, Z)
#     return zeros(eltype(Z), 0)
    # N = params.N
    # idx = params.idx
    # c = zeros(eltype(Z), params.N*2)
    # world = RigidBodyDynamics.findbody(bb_arms.mech, "world")
    # for i =1:N
    #     state = MechanismState(params.model1.mech)

    #     q = zeros(9)
    #     q[1] = -5.0 # move to a distance
    #     q[3] = -0.15 # lean forward
    #     q[6] = 0.785 # right shoulder
    #     q[7] = -0.785 # left shoulder
    #     q[8] = 1.5708 # right elbow 90 deg
    #     q[9] = -1.5708 # left elbow 90 deg
    #     RigidBodyDynamics.set_configuration!(state, Z[idx.x[i][1:9]])
    #     RigidBodyDynamics.set_velocity!(contact_state, zeros(9))

    #     right_end = RigidBodyDynamics.findbody(bb_arms.mech, "RArm4")
    #     right_end_tf = RigidBodyDynamics.frame_definitions(right_end)[end].from
    #     r_end_eff_pos = transform(contact_state, Point3D(right_end_tf, zero(SVector{3})), default_frame(world))
    #     c[2*i-1] = r_end_eff_pos.v[2]

    #     left_end = RigidBodyDynamics.findbody(bb_arms.mech, "LArm4")
    #     left_end_tf = RigidBodyDynamics.frame_definitions(left_end)[end].from
    #     l_end_eff_pos = transform(contact_state, Point3D(left_end_tf, zero(SVector{3})), default_frame(world))
    #     c[2*i] = l_end_eff_pos.v[2]
    # end
    return centroidal_inequality_constraint(params, Z)
end

In [2]:
# Optimization Parameters
h = 0.2 # 5 Hz
nq = 9
nx = nq*2+1     # number of state
nu = 7     # number of controls
Tfinal = 4.0 # final time
Nt = Int(Tfinal/h)   # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));
# n_nlp = (nx+nu)*Nt # number of decision variables
# m_nlp = nx*(Nt+1)

# Cost terms
diagq = ones(nx)
diagq[1:2] = diagq[1:2]*1 # position cost
diagq[3:4] = diagq[3:4]*1 # lean angles cost

# Cost matrices
Q = diagm(diagq)
Qf = 10*Q
R = 0.5*diagm(ones(nu))


xic = zeros(nx)
#!!!!!!!!!!!!!!!!!!!!!add all the goal states!!!!!!!!!!!!!!!!!!!!
nh = nḣ = nr = nṙ = nr̈ = nF = nτ = ntd = 3
nd = 6
nfriction = 1
μ = 0.5
Qh = diagm(ones(nh))
Qa = diagm(ones(nx))
Qcom = diagm(ones(nh))
Qps = diagm(ones(2))*2
Qvs = diagm(ones(2))*10
Qfv = diagm(ones(2))*10
Qfps = Qps*10
Qfh = diagm(ones(nh))*10
Qfḣ = diagm(ones(nh))*10

vs_g = zeros(2)
ps_g = zeros(2)
ps_g[2] = r_end_eff_pos.v[2]/2
n = zeros(3)
n[2] = -1
m = mass(bb_wall.mech)
g = [0.0, 0.0, -9.81];
qic = zeros(nx)
q̇ic = zeros(nx) 
q̈ic = zeros(nx)
q̇ic[2] = 0.5
# add all the Q, goal states, normal vector, slipping constant
# add all the Q, goal states, normal vector, slipping constant, r_end_eff_pos
# xic = [q; zeros(9)]
xg = zeros(nx)
# xg = [q; zeros(9)]

# Indexing 
idx = create_idx(nx,Nt);

diff_type = :auto

# Create warm start trajectory
q = zeros(19)
q[1] = -0.5 # move to a distance
q[3] = -0.10 # lean forward
q[6] = 0.785 # right shoulder
q[7] = -0.785 # left shoulder
q[12] = 1.5708 # right elbow 90 deg
q[13] = -1.5708 # left elbow 90 deg


Xref = [zeros(nx) for i=1:Nt]
Xfwd = collect(range(xic, q, length = 10))
Xwall = [q; zeros(9)]
Xbwd = collect(range(q, xg, length = 10))
for i=1:10
    Xref[i] .= Xfwd[i]
end
# for i=1:5
#     Xref[i] .= Xwall
# end
for i=11:20
    Xref[i] .= Xbwd[i-10]
end
Uref = [zeros(7) for i = 1:(Nt-1)];

# Contact scheduling
M1 = [1:8; 13:20]
# M2 = [11:15;]
M2 = []
J1 = [10:12]
J2 = []
n = [0;-1;0]
# Models
model1 = Ballbot_arms()
model2 = Ballbot_wall(vec(l_end_eff_pos.v), vec(r_end_eff_pos.v))

# Create params
params = (xic = xic,
          xg = xg,
          xcgs = [q],
          Xref = Xref,
          Uref = Uref,
          dt = h,
          N = Nt,
          idx = idx,
          Q = Q,
          R = R,
          Qf = Qf,
          M1 = M1, 
          M2 = M2,
          J1 = J1, 
          J2 = J2,
          model1 = model1,
          model2 = model2,
          μ = μ,
          nx = nx,
          nu = nu,
          vs_g=vs_g,
          ps_g = ps_g,
          Qh = diagm(ones(nh)),
          Qa = Qa,
          Qcom = Qcom,
          Qps = Qps,
          Qvs = Qvs,
          Qfv = Qfv,
          Qfps = Qfps,
          Qfh = Qfh,
          Qfḣ = Qfḣ,
          r_end_eff_pos = r_end_eff_pos,
          l_end_eff_pos = l_end_eff_pos,
          m=m,
          g=g,
          qic = qic,
          q̇ic = q̇ic, 
          q̈ic = q̈ic,
          n=n,
          );

# Primal bounds TODO chnage the bounds
x_l = ones(idx.nz)
x_u = ones(idx.nz)

#limits to states TODO make this full state
stateq_l = [
    -2.0, -0.1, # limit the ballbot's movement
    -0.349065850399, -0.349065850399, -3.15, # from manufacturer
    -2.96705972839, -2.96705972839, # from manufacturer
    -0.174        , -3.31613,
    -2.96705972839, -2.96705972839 ,
    -0.349065850399, -2.79252680319, # from manufacturer
    -2.96705972839, -2.96705972839 ,
    -1.57079        , -1.57079     ,
    -1.57079        , -1.57079     ,
    ]
stateq_u = [
    2.0, 0.1,
    0.349065850399, 0.349065850399, 3.15,
    2.96705972839, 2.96705972839,
    3.31613      , 0.174,
    2.96705972839, 2.96705972839 ,
    2.79252680319, 0.349065850399,
    2.96705972839, 2.96705972839 ,
    1.57079        , 1.57079   ,
    1.57079        , 1.57079     ,
    ]

statedq_l = ones(nx)*-50
statedq_u = ones(nx)*50
stateddq_l = ones(nx)*-10
stateddq_u = ones(nx)*10

statecom_l = ones(nr)*-10
statecom_u = ones(nr)*10
statedcom_l = ones(nr)*-5
statedcom_u = ones(nr)*5
stateddcom_l = ones(nr)*-10
stateddcom_u = ones(nr)*10

stateh_l = ones(nh)*-200
stateh_u = ones(nh)*200
statedh_l = ones(nh)*-200
statedh_u = ones(nh)*200

stateFb_l = ones(nF)*-Inf
stateFb_u = ones(nF)*Inf
stateFL_l = ones(nF)*-60
stateFL_u = ones(nF)*60
stateFR_l = ones(nF)*-60
stateFR_u = ones(nF)*60

stateτb_l = ones(nτ)*-Inf
stateτb_u = ones(nτ)*Inf
stateτL_l = ones(nτ)*-500
stateτL_u = ones(nτ)*500
stateτR_l = ones(nτ)*-500
stateτR_u = ones(nτ)*500

statedL_l = ones(nd)*-Inf
statedL_u = zeros(nd)
statedR_l = ones(nd)*-Inf
statedR_u = zeros(nd)

# state_l = vcat(stateq_l,statedq_l,stateddq_l,statecom_l,statedcom_l,stateddcom_l,stateFb_l,stateFL_l,stateFR_l,stateτb_l,stateτL_l,stateτR_l,statedL_l,statedR_l)
# state_u = vcat(stateq_u,statedq_u,stateddq_u,statecom_u,statedcom_u,stateddcom_u,stateFb_u,stateFL_u,stateFR_u,stateτb_u,stateτL_u,stateτR_u,statedL_u,statedR_u)

# control_l = -30 * ones(7)
# control_u = 30 * ones(7)




for i = 1:(Nt)
    x_l[idx.q[i]] .= stateq_l
    x_u[idx.q[i]] .= stateq_u
    x_l[idx.q̇[i]] .= statedq_l
    x_u[idx.q̇[i]] .= statedq_u
    x_l[idx.q̈[i]] .= stateddq_l
    x_u[idx.q̈[i]] .= stateddq_u
    x_l[idx.r[i]] .= statecom_l
    x_u[idx.r[i]] .= statecom_u
    x_l[idx.ṙ[i]] .= statedcom_l
    x_u[idx.ṙ[i]] .= statedcom_u
    x_l[idx.r̈[i]] .= stateddcom_l
    x_u[idx.r̈[i]] .= stateddcom_u
    x_l[idx.h[i]] .= stateh_l
    x_u[idx.h[i]] .= stateh_u
    x_l[idx.ḣ[i]] .= statedh_l
    x_u[idx.ḣ[i]] .= statedh_u
    x_l[idx.Fb[i]] .= stateFb_l
    x_u[idx.Fb[i]] .= stateFb_u
    x_l[idx.FL[i]] .= stateFL_l
    x_u[idx.FL[i]] .= stateFL_u
    x_l[idx.FR[i]] .= stateFR_l
    x_u[idx.FR[i]] .= stateFR_u
    x_l[idx.τb[i]] .= stateτb_l
    x_u[idx.τb[i]] .= stateτb_u
    x_l[idx.τL[i]] .= stateτL_l
    x_u[idx.τL[i]] .= stateτL_u
    x_l[idx.τR[i]] .= stateτR_l
    x_u[idx.τR[i]] .= stateτR_u
    x_l[idx.dL[i]] .= statedL_l
    x_u[idx.dL[i]] .= statedL_u
    x_l[idx.dR[i]] .= statedR_l
    x_u[idx.dR[i]] .= statedR_u
end

# x_l[idx.x[end]] .= state_l
# x_u[idx.x[end]] .= state_u

# TODO Inequality constraint bounds (this is what we do when we have no inequality constraints)
c_l = ones(idx.ncineq)
c_u = ones(idx.ncineq)
for i = 1:(Nt)
    c_l[idx.gcfL[i]] *= -Inf
    c_u[idx.gcfL[i]] *= 0.0
    c_l[idx.gcfR[i]] *= -Inf
    c_u[idx.gcfR[i]] *= 0.0
end

verbose=true

z0 = zeros(idx.nz)
# TODO fix this
for i in 1:(Nt)
    z0[idx.q[i]] .= Xref[i]
    # 0.001*randn(idx.nu)
    z0[idx.q̇[i]] = 0.001*randn(idx.nq)
    z0[idx.q̈[i]] = 0.001*randn(idx.nq)
    z0[idx.r[i]] = 0.001*randn(idx.nr)
    z0[idx.ṙ[i]] = 0.001*randn(idx.nṙ)
    z0[idx.r̈[i]] = 0.001*randn(idx.nr̈)
    z0[idx.h[i]] = 0.001*randn(idx.nh)
    z0[idx.ḣ[i]] = 0.001*randn(idx.nḣ)
    z0[idx.Fb[i]] = 0.001*randn(idx.nF)
    z0[idx.FL[i]] = 0.001*randn(idx.nF)
    z0[idx.FR[i]] = 0.001*randn(idx.nF)
    z0[idx.τb[i]] = 0.001*randn(idx.nτ)
    z0[idx.τL[i]] = 0.001*randn(idx.nτ)
    z0[idx.τR[i]] = 0.001*randn(idx.nτ)
    z0[idx.dL[i]] = 0.001*randn(idx.nd)
    z0[idx.dR[i]] = 0.001*randn(idx.nd)
end

Z = fmincon(ballbot_cost,centroidal_equality_constraints,centroidal_inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 100, verbose = verbose);


LoadError: UndefVarError: diagm not defined

In [ ]:
X = [Z[idx.q[i]] for i = 1:Nt]

X1 = [SVector{18}(x[1:18]) for x in X];
vis = Visualizer()
display(render(vis))
mvis = MechanismVisualizer(ballbot.mech, URDFVisuals(urdf_arms), vis["ballbot"])
animation = mc.Animation(mvis, thist, X)
setanimation!(mvis, animation)

box = HyperRectangle(Vec(-2.5, r_end_eff_pos.v[2], 0), Vec(5., 0.1, 5))
setobject!(vis["wall"], box)


In [ ]:
@show [Xi[1:9] for Xi in X]
# @show [Xi[10:end] for Xi in X]

In [ ]:
@show [q; zeros(9)]

In [ ]:
xi = Z[idx.x[1]]
ui = Z[idx.u[1]]
rk4(params, params.model1, xi, ui, h)

In [ ]:
xg